In [25]:
import cv2
import numpy as np
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import cascaded_union
import random

no_of_houses = 10
no_of_valleys = 2
no_of_hills = 2
house_width, house_height = 20, 20
building_width, building_height = random.randint(30, 60), random.randint(30, 60)
hill_radius = 30
valley_radius = 20
building_labels = ["Hospital", "Gas Station", "Public Park","School"]

def random_non_intersecting_polygon(existing_polygons, width, height):
    while True:
        x, y = random.randint(0, 512 - width), random.randint(0, 512 - height)
        new_polygon = Polygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
        
        intersects = False
        for existing_polygon in existing_polygons:
            if new_polygon.intersects(existing_polygon):
                intersects = True
                break
        
        if not intersects:
            return new_polygon

# Initialize the image
image = np.zeros((512, 512), dtype=np.uint8)

# Draw a hill
for _ in range(no_of_hills):
    hill_center = (random.randint(hill_radius, 512 - hill_radius), random.randint(hill_radius, 512 - hill_radius))
    cv2.circle(image, hill_center, hill_radius, 255, -1)
    # hill_polygon = Polygon(hill_center).buffer(hill_radius)
    hill_polygon = Polygon([(hill_center[0] + hill_radius * np.cos(angle), hill_center[1] + hill_radius * np.sin(angle)) for angle in np.linspace(0, 2 * np.pi, 100)]).buffer(0)


# Draw valleys
valley_polygons = []
for _ in range(no_of_valleys):
    x, y = random.randint(0, 512 - 1), random.randint(0, 512 - 1)
    valley_polygon = Polygon([(x, y), (x + 1, y), (x + 1, y + 1), (x, y + 1)]).buffer(valley_radius)
    valley_polygons.append(valley_polygon)

    for x, y in valley_polygon.exterior.coords:
        cv2.circle(image, (int(x), int(y)), 1, 255, -1)

# Draw houses
house_polygons = []
for _ in range(no_of_houses):
    house_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], house_width, house_height)
    house_polygons.append(house_polygon)

    x, y = int(house_polygon.bounds[0]), int(house_polygon.bounds[1])
    cv2.rectangle(image, (x, y), (x + house_width, y + house_height), 255, -1)

# Draw hospital, gas station, and public park

for label in building_labels:
    building_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], building_width, building_height)
    house_polygons.append(building_polygon)

    x, y = int(building_polygon.bounds[0]), int(building_polygon.bounds[1])
    cv2.rectangle(image, (x, y), (x + building_width, y + building_height), 255, -1)
    cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

import networkx as nx

def get_polygon_center(polygon):
    x, y = np.array(polygon.centroid.coords).squeeze()
    return int(x), int(y)

#Extract center points of shapes
center_points = []
for polygon in house_polygons:
    center_points.append(get_polygon_center(polygon))

# Create a complete graph using the NetworkX library
G = nx.complete_graph(len(center_points))

# Set the node positions to the center points of the shapes
pos = {i: center_points[i] for i in range(len(center_points))}
nx.set_node_attributes(G, pos, "pos")

# Visualize the graph
image_with_graph = image.copy()
for i, j in G.edges():
    cv2.line(image_with_graph, center_points[i], center_points[j], 128, 1)

from shapely.geometry import LineString
import math

# Check if a line segment intersects with a polygon
def line_intersects_polygon(line_start, line_end, polygon):
    line = LineString([line_start, line_end])
    return line.intersects(polygon)

# Calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def edge_weight(p1, p2, hill_polygon, valley_polygons):
    weight = euclidean_distance(p1, p2)
    line = LineString([p1, p2])
    
    if line.intersects(hill_polygon):
        weight *= 2
    
    if any(line.intersects(valley) for valley in valley_polygons):
        weight *= 3

    return weight

# Add edge weights based on the distance and the presence of hills or valleys
for i, j in G.edges():
    G[i][j]['weight'] = edge_weight(center_points[i], center_points[j], hill_polygon, valley_polygons)

# Visualize the graph with edge weights
image_with_graph_weights = image.copy()
for i, j in G.edges():
    cv2.line(image_with_graph_weights, center_points[i], center_points[j], 128, 1)
    
    # Calculate the midpoint of the edge
    midpoint = ((center_points[i][0] + center_points[j][0]) // 2, (center_points[i][1] + center_points[j][1]) // 2)
    
    # Draw the edge weight as a text label
    weight = int(G[i][j]['weight'])
    cv2.putText(image_with_graph_weights, str(weight), midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

cv2.imshow("Graph with Weights", image_with_graph_weights)
cv2.waitKey(0)
cv2.destroyAllWindows()

# cv2.imwrite("graph_with_weights.png", image_with_graph_weights)


In [39]:
import cv2
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, LineString
from shapely.ops import cascaded_union
import random
import math

no_of_houses = 10
no_of_valleys = 2
no_of_hills = 2
house_width, house_height = 20, 20
building_width, building_height = random.randint(30, 60), random.randint(30, 60)
hill_radius = 30
valley_radius = 20
building_labels = ["Hospital", "Gas Station", "Public Park", "School"]

def random_non_intersecting_polygon(existing_polygons, width, height):
    while True:
        x, y = random.randint(0, 512 - width), random.randint(0, 512 - height)
        new_polygon = Polygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
        
        intersects = False
        for existing_polygon in existing_polygons:
            if new_polygon.intersects(existing_polygon):
                intersects = True
                break
        
        if not intersects:
            return new_polygon

# Initialize the image
image = np.zeros((512, 512), dtype=np.uint8)

# Draw a hill
for _ in range(no_of_hills):
    hill_center = (random.randint(hill_radius, 512 - hill_radius), random.randint(hill_radius, 512 - hill_radius))
    cv2.circle(image, hill_center, hill_radius, 255, -1)
    hill_polygon = Polygon([(hill_center[0] + hill_radius * np.cos(angle), hill_center[1] + hill_radius * np.sin(angle)) for angle in np.linspace(0, 2 * np.pi, 100)]).buffer(0)

# Draw valleys
valley_polygons = []
for _ in range(no_of_valleys):
    x, y = random.randint(0, 512 - 1), random.randint(0, 512 - 1)
    valley_polygon = Polygon([(x, y), (x + 1, y), (x + 1, y + 1), (x, y + 1)]).buffer(valley_radius)
    valley_polygons.append(valley_polygon)

    for x, y in valley_polygon.exterior.coords:
        cv2.circle(image, (int(x), int(y)), valley_radius, 255, -1)

# Draw houses
house_polygons = []
for _ in range(no_of_houses):
    house_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], house_width, house_height)
    house_polygons.append(house_polygon)

    x, y = int(house_polygon.bounds[0]), int(house_polygon.bounds[1])
    cv2.rectangle(image, (x, y), (x + house_width, y + house_height), 255, -1)

# Draw buildings with labels
label_polygons = []
for label in building_labels:
    building_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], building_width, building_height)
    label_polygons.append((building_polygon, label))

    x, y = int(building_polygon.bounds[0]), int(building_polygon.bounds[1])
    cv2.rectangle(image, (x, y), (x + building_width, y + building_height), 255, -1)
    cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

def get_polygon_center(polygon):
    x, y = np.array(polygon.centroid.coords).squeeze()
    return int(x), int(y)

#Extract center points of shapes
center_points = []
for polygon in house_polygons:
    center_points.append(get_polygon_center(polygon))

# Add center points for labeled buildings
for polygon, label in label_polygons:
    center_points.append(get_polygon_center(polygon))

# Create a complete graph using the NetworkX library
G = nx.complete_graph(len(center_points))

# Set the node positions to the center points of the shapes
pos = {i: center_points[i] for i in range(len(center_points))}
nx.set_node_attributes(G, pos, "pos")

# Visualize the graph
image_with_graph = image.copy()
for i, j in G.edges():
    cv2.line(image_with_graph, center_points[i], center_points[j], 128, 1)

# Check if a line segment intersects with a polygon
def line_intersects_polygon(line_start, line_end, polygon):
    line = LineString([line_start, line_end])
    return line.intersects(polygon)

# Calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def edge_weight(p1, p2, hill_polygon, valley_polygons):
    weight = euclidean_distance(p1, p2)
    line = LineString([p1, p2])
    
    if line.intersects(hill_polygon):
        weight *= 2
    
    if any(line.intersects(valley) for valley in valley_polygons):
        weight *= 3

    return weight

# Add edge weights based on the distance and the presence of hills or valleys
for i, j in G.edges():
    G[i][j]['weight'] = edge_weight(center_points[i], center_points[j], hill_polygon, valley_polygons)

# Visualize the graph with edge weights
image_with_graph_weights = image.copy()
for i, j in G.edges():
    cv2.line(image_with_graph_weights, center_points[i], center_points[j], 128, 1)
    
    # Calculate the midpoint of the edge
    midpoint = ((center_points[i][0] + center_points[j][0]) // 2, (center_points[i][1] + center_points[j][1]) // 2)
    
    # Draw the edge weight as a text label
    weight = int(G[i][j]['weight'])
    cv2.putText(image_with_graph_weights, str(weight), midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)


# Assign vertex weights
# vertex_weights = {}
# for i, (polygon, label) in enumerate(label_polygons):
#     vertex_weights[len(house_polygons) + i] = {"Hospital": 4, "Gas Station": 3, "Public Park": 2, "School": 1}[label]

# print(vertex_weights)

# # Set vertex weights for each node
# for idx, (polygon, label) in enumerate(house_polygons):
#     vertex_weight = vertex_weights[label]
#     G.nodes[idx]['vertex_weight'] = vertex_weight

# # Display vertex weights
# for idx, (polygon, label) in enumerate(house_polygons):
#     vertex_weight = G.nodes[idx]['vertex_weight']
#     vertex_center = get_polygon_center(polygon)
#     cv2.putText(image_with_graph_weights, str(vertex_weight), vertex_center, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0, 1)


# Visualize the graph with edge weights and vertex weights
# image_with_graph_weights = image.copy()
# for i, j in G.edges():
#     cv2.line(image_with_graph_weights, center_points[i], center_points[j], 128, 1)
    
#     # Calculate the midpoint of the edge
#     midpoint = ((center_points[i][0] + center_points[j][0]) // 2, (center_points[i][1] + center_points[j][1]) // 2)
    
#     # Draw the edge weight as a text label
#     weight = int(G[i][j]['weight'])
#     cv2.putText(image_with_graph_weights, str(weight), midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

# # Draw the vertex weights on top of the vertex
# for i, pos in nx.get_node_attributes(G, "pos").items():
#     if i in vertex_weights:
#         weight = vertex_weights[i]
#         cv2.putText(image_with_graph_weights, str(weight), pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)


cv2.imshow("Graph with Weights", image_with_graph_weights)
cv2.waitKey(0)
cv2.destroyAllWindows()

# cv2.imwrite("graph_with_weights.png", image_with_graph_weights)


In [53]:
import cv2
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, LineString
from shapely.ops import cascaded_union
import random
import math
import networkx as nx

no_of_houses = 20
no_of_valleys = 3
no_of_hills = 4
house_width, house_height = 20, 20
building_width, building_height = random.randint(30, 60), random.randint(30, 60)
hill_radius = 30
valley_radius = 20
building_labels = ["Hospital", "Gas Station", "Public Park", "School"]

vertex_weights = {"House": 1, "Hospital": 2, "Gas Station": 3, "Public Park": 4, "School": 5}

def random_non_intersecting_polygon(existing_polygons, width, height):
    while True:
        x, y = random.randint(0, 512 - width), random.randint(0, 512 - height)
        new_polygon = Polygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
        
        intersects = False
        for existing_polygon in existing_polygons:
            if new_polygon.intersects(existing_polygon):
                intersects = True
                break
        
        if not intersects:
            return new_polygon

# Initialize the image
image = np.zeros((512, 512), dtype=np.uint8)

# Draw a hill
for _ in range(no_of_hills):
    hill_center = (random.randint(hill_radius, 512 - hill_radius), random.randint(hill_radius, 512 - hill_radius))
    cv2.circle(image, hill_center, hill_radius, 255, -1)
    hill_polygon = Polygon([(hill_center[0] + hill_radius * np.cos(angle), hill_center[1] + hill_radius * np.sin(angle)) for angle in np.linspace(0, 2 * np.pi, 100)]).buffer(0)

# Draw valleys
valley_polygons = []
for _ in range(no_of_valleys):
    x, y = random.randint(0, 512 - 1), random.randint(0, 512 - 1)
    valley_polygon = Polygon([(x, y), (x + 1, y), (x + 1, y + 1), (x, y + 1)]).buffer(valley_radius)
    valley_polygons.append(valley_polygon)

    for x, y in valley_polygon.exterior.coords:
        cv2.circle(image, (int(x), int(y)), valley_radius, 255, -1)

# Draw houses
house_polygons = []
for _ in range(no_of_houses):
    house_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], house_width, house_height)
    house_polygons.append(house_polygon)

    x, y = int(house_polygon.bounds[0]), int(house_polygon.bounds[1])
    cv2.rectangle(image, (x, y), (x + house_width, y + house_height), 255, -1)

# Draw buildings with labels
label_polygons = []
for label in building_labels:
    building_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], building_width, building_height)
    label_polygons.append((building_polygon, label))

    x, y = int(building_polygon.bounds[0]), int(building_polygon.bounds[1])
    cv2.rectangle(image, (x, y), (x + building_width, y + building_height), 255, -1)
    cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

def get_polygon_center(polygon):
    x, y = np.array(polygon.centroid.coords).squeeze()
    return int(x), int(y)

#Extract center points of shapes
center_points = []
for polygon in house_polygons:
    center_points.append(get_polygon_center(polygon))

# Add center points for labeled buildings
for polygon, label in label_polygons:
    center_points.append(get_polygon_center(polygon))

# Create a complete graph using the NetworkX library
G = nx.complete_graph(len(center_points))

# Set the node positions to the center points of the shapes
pos = {i: center_points[i] for i in range(len(center_points))}
nx.set_node_attributes(G, pos, "pos")

# Visualize the graph
image_with_graph = image.copy()
for i, j in G.edges():
    cv2.line(image_with_graph, center_points[i], center_points[j], 128, 1)

# Check if a line segment intersects with a polygon
def line_intersects_polygon(line_start, line_end, polygon):
    line = LineString([line_start, line_end])
    return line.intersects(polygon)

# Calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def edge_weight(p1, p2, hill_polygon, valley_polygons):
    weight = euclidean_distance(p1, p2)
    line = LineString([p1, p2])
    
    if line.intersects(hill_polygon):
        weight *= 2
    
    if any(line.intersects(valley) for valley in valley_polygons):
        weight *= 3

    return weight

# Add edge weights based on the distance and the presence of hills or valleys
for i, j in G.edges():
    G[i][j]['weight'] = edge_weight(center_points[i], center_points[j], hill_polygon, valley_polygons)

# Visualize the graph with edge weights
image_with_graph_weights = image.copy()
for i, j in G.edges():
    cv2.line(image_with_graph_weights, center_points[i], center_points[j], 128, 1)
    
    # Calculate the midpoint of the edge
    midpoint = ((center_points[i][0] + center_points[j][0]) // 2, (center_points[i][1] + center_points[j][1]) // 2)
    
    # Draw the edge weight as a text label
    weight = int(G[i][j]['weight'])
    cv2.putText(image_with_graph_weights, str(weight), midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)
    
# Set the node positions to the center points of the shapes
pos = {i: center_points[i] for i in range(len(center_points))}
nx.set_node_attributes(G, pos, "pos")

# Assign vertex weights based on the type of building
vertex_weights_list = [vertex_weights["House"]] * no_of_houses
for _, label in label_polygons:
    vertex_weights_list.append(vertex_weights[label])
nx.set_node_attributes(G, {i: vertex_weights_list[i] for i in range(len(center_points))}, "weight")

# Visualize the graph with vertex weights
image_with_vertex_weights = image_with_graph_weights.copy()
for i in range(len(center_points)):
    x, y = center_points[i]
    weight = int(G.nodes[i]['weight'])
    cv2.putText(image_with_vertex_weights, str(weight), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0, 1)

cv2.imshow("Graph with Vertex Weights", image_with_vertex_weights)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [57]:
# import networkx as nx

# # Find the minimum spanning tree
# mst = nx.minimum_spanning_tree(G, algorithm='kruskal', weight='weight')

# # Find the shortest paths between all pairs of nodes
# shortest_paths = {}
# for i in mst.nodes():
#     for j in mst.nodes():
#         if i != j:
#             path = nx.shortest_path(mst, i, j, weight='weight')
#             shortest_paths[(i, j)] = path

# # Visualize the minimum spanning tree
# image_with_mst = image.copy()
# for i, j in mst.edges():
#     cv2.line(image_with_mst, center_points[i], center_points[j], 128, 1)

# cv2.imshow("Minimum Spanning Tree", image_with_mst)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# # Visualize the shortest paths
# image_with_shortest_paths = image.copy()
# for path in shortest_paths.values():
#     for i in range(len(path) - 1):
#         cv2.line(image_with_shortest_paths, center_points[path[i]], center_points[path[i + 1]], 128, 1)

# cv2.imshow("Shortest Paths", image_with_shortest_paths)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [55]:
def kruskal_with_vertex_weights(graph):
    sorted_edges = sorted(graph.edges(data=True), key=lambda x: x[2]['weight'] + graph.nodes[x[0]]['weight'] + graph.nodes[x[1]]['weight'])
    mst = nx.Graph()

    for u, v, data in sorted_edges:
        if u not in mst or v not in mst or not nx.has_path(mst, u, v):
            mst.add_edge(u, v, weight=data['weight'])

    return mst

# Find the Minimum Spanning Tree
mst = kruskal_with_vertex_weights(G)

# Visualize the MST
image_with_mst = image.copy()
for i, j in mst.edges():
    cv2.line(image_with_mst, center_points[i], center_points[j], 128, 2)

    # Calculate the midpoint of the edge
    midpoint = ((center_points[i][0] + center_points[j][0]) // 2, (center_points[i][1] + center_points[j][1]) // 2)
    
    # Draw the edge weight as a white text label
    weight = int(mst[i][j]['weight'])
    cv2.putText(image_with_mst, str(weight), midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

for i in range(len(center_points)):
    x, y = center_points[i]
    
    # Draw the vertex weight as a black text label
    weight = int(G.nodes[i]['weight'])
    cv2.putText(image_with_mst, str(weight), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0, 1)

cv2.imshow("Minimum Spanning Tree", image_with_mst)
cv2.waitKey(0)
cv2.destroyAllWindows()
